<a href="https://colab.research.google.com/github/JanithShimalka/Colaboratory/blob/main/AIML_Pro_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
data = pd.read_csv('Tweets.csv')

In [22]:
data.shape

(14640, 15)

In [3]:
data.head(5)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [4]:
df = data[["airline_sentiment", "text"]]



In [21]:
df.shape

(14640, 3)

In [5]:
df.sample(5)

,airline_sentiment,text
8314,negative,"@JetBlue I want to add, my flight to Mexico wa..."
6625,negative,"@SouthwestAir not terribly inspired to call ""c..."
11583,negative,@USAirways I absolutely knew you would try to ...
9266,negative,@USAirways I agree but per the captain this is...
10882,negative,"@USAirways get ur act together, start treating..."


In [7]:
import nltk

import string

import re



from nltk.stem.porter import PorterStemmer



nltk.download('stopwords')

from nltk.corpus import stopwords



nltk.download('punkt')

ps = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
def clean_text(text):

    text = text.lower()

    text = re.sub(r'http.?://[^\s]+[\s]?', '', text)

    text = nltk.word_tokenize(text)

    y = []

    for i in text:

      if i not in stopwords.words('english'):

            y.append(i)

    text = y[:]

    y.clear()

    for i in text:

        y.append(ps.stem(i))

    return " ".join(y)

In [9]:
df['text_cleaned']= df['text'].apply(clean_text)

<ipython-input-9-f480e08ddc7e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_cleaned']= df['text'].apply(clean_text)


In [10]:
df.head(5)

,airline_sentiment,text,text_cleaned
0,neutral,@VirginAmerica What @dhepburn said.,@ virginamerica @ dhepburn said .
1,positive,@VirginAmerica plus you've added commercials t...,@ virginamerica plu 've ad commerci experi ......
2,neutral,@VirginAmerica I didn't today... Must mean I n...,@ virginamerica n't today ... must mean need t...
3,negative,@VirginAmerica it's really aggressive to blast...,@ virginamerica 's realli aggress blast obnoxi...
4,negative,@VirginAmerica and it's a really big bad thing...,@ virginamerica 's realli big bad thing


In [20]:
x.shape

(14640, 3000)

In [25]:
df.groupby('airline_sentiment').describe()

text         \
                  count unique   
airline_sentiment                
negative           9178   9087   
neutral            3099   3067   
positive           2363   2298   

                                                                           \
                                                                 top freq   
airline_sentiment                                                           
negative           @AmericanAir that's 16+ extra hours of travel ...    2   
neutral                                           @SouthwestAir sent    5   
positive                                            @JetBlue thanks!    5   

                  text_cleaned         \
                         count unique   
airline_sentiment                       
negative                  9178   9083   
neutral                   3099   3025   
positive                  2363   2262   

                                                                           
                                                                 top freq  
airline_sentiment                                                          
negative           @ americanair fyi ... call still get drop . ho...    2  
neutral            @ jetblu 's ceo battl appeas passeng wall stre...    8  
positive                                            @ jetblu thank !   12

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_vec = TfidfVectorizer(max_features = 3000)
x = tf_vec.fit_transform(df['text_cleaned']).toarray()

In [12]:
y = df['airline_sentiment'].values

In [19]:
x.shape

(14640, 3000)

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 2)


In [14]:
from sklearn.naive_bayes import MultinomialNB
modelNB = MultinomialNB()
modelNB.fit(x_train,y_train)


MultinomialNB()

In [16]:
from sklearn.metrics import accuracy_score
NBpred = modelNB.predict(x_test)
print(accuracy_score(y_test,NBpred))

0.7219945355191257


In [18]:
from sklearn.ensemble import RandomForestClassifier



model = RandomForestClassifier()

model.fit(x_train,y_train)

y_pred = model.predict(x_test)

print(accuracy_score(y_test,y_pred))

0.7523907103825137
